In [1]:
#We import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
#We adjust the path
path = "C:/Users/lunam/OzonProject/"

In [32]:
#We import the forecasting dataset
df_sta_for = pd.read_csv(path + "sta_data_forecasting.csv", low_memory=False)
df_sta_for

,Unnamed: 0,Luftdruck_STA,Lufttemp_STA,Windgeschw_STA,StampfenbachstrOzon,AnzFahrzeuge_STA,hour_of_day,day_of_week,day_of_year,month,year
0,2021-01-01 01:00:00,954.0,2.1,4.7,8.51,1320.0,1,5,1,1,2021
1,2021-01-01 02:00:00,954.4,2.3,3.2,5.34,919.0,2,5,1,1,2021
2,2021-01-01 03:00:00,954.6,2.0,2.5,7.54,617.0,3,5,1,1,2021
3,2021-01-01 04:00:00,954.6,2.3,2.2,31.30,445.0,4,5,1,1,2021
4,2021-01-01 05:00:00,954.8,2.3,2.9,40.87,371.0,5,5,1,1,2021
...,...,...,...,...,...,...,...,...,...,...,...
25554,2023-12-01 19:00:00,955.2,3.2,5.8,NaN,3215.0,19,5,335,12,2023
25555,2023-12-01 20:00:00,955.0,3.2,7.2,NaN,2477.0,20,5,335,12,2023
25556,2023-12-01 21:00:00,954.5,3.0,6.8,NaN,2115.0,21,5,335,12,2023
25557,2023-12-01 22:00:00,954.0,2.9,5.4,NaN,1865.0,22,5,335,12,2023


In [4]:
#We change the name of the datetime column
df_sta_for = df_sta_for.rename(columns={'Unnamed: 0': 'datetime'})
df_sta_for

,datetime,Luftdruck_STA,Lufttemp_STA,Windgeschw_STA,StampfenbachstrOzon,AnzFahrzeuge_STA,hour_of_day,day_of_week,day_of_year,month,year
0,2021-01-01 01:00:00,954.0,2.1,4.7,8.51,1320.0,1,5,1,1,2021
1,2021-01-01 02:00:00,954.4,2.3,3.2,5.34,919.0,2,5,1,1,2021
2,2021-01-01 03:00:00,954.6,2.0,2.5,7.54,617.0,3,5,1,1,2021
3,2021-01-01 04:00:00,954.6,2.3,2.2,31.30,445.0,4,5,1,1,2021
4,2021-01-01 05:00:00,954.8,2.3,2.9,40.87,371.0,5,5,1,1,2021
...,...,...,...,...,...,...,...,...,...,...,...
25554,2023-12-01 19:00:00,955.2,3.2,5.8,NaN,3215.0,19,5,335,12,2023
25555,2023-12-01 20:00:00,955.0,3.2,7.2,NaN,2477.0,20,5,335,12,2023
25556,2023-12-01 21:00:00,954.5,3.0,6.8,NaN,2115.0,21,5,335,12,2023
25557,2023-12-01 22:00:00,954.0,2.9,5.4,NaN,1865.0,22,5,335,12,2023


In [5]:
#We set the column 'datetime' as index
df_sta_for['datetime'] = pd.to_datetime(df_sta_for['datetime'])
df_sta_for.set_index('datetime', inplace=True)

In [6]:
#We fill the NaN values from the forecasting day with 0.0 in order to avoid problems later
df_sta_for = df_sta_for.fillna(0.0)

In [7]:
df_sta_for.head()

,Luftdruck_STA,Lufttemp_STA,Windgeschw_STA,StampfenbachstrOzon,AnzFahrzeuge_STA,hour_of_day,day_of_week,day_of_year,month,year
datetime,,,,,,,,,,
2021-01-01 01:00:00,954.0,2.1,4.7,8.51,1320.0,1,5,1,1,2021
2021-01-01 02:00:00,954.4,2.3,3.2,5.34,919.0,2,5,1,1,2021
2021-01-01 03:00:00,954.6,2.0,2.5,7.54,617.0,3,5,1,1,2021
2021-01-01 04:00:00,954.6,2.3,2.2,31.30,445.0,4,5,1,1,2021
2021-01-01 05:00:00,954.8,2.3,2.9,40.87,371.0,5,5,1,1,2021


In [8]:
test = df_sta_for[-8664:] # get last year
train = df_sta_for[0:-8664] # get data until first day of testing day

In [9]:
shifts = np.arange(24, 73).astype(int) #we set lags from 24 to 72 (1 day and 3 days)

In [10]:
#We create the lag values
shifted_data = {"lag_{}_hour".format(hour_shift): train["StampfenbachstrOzon"].shift(hour_shift) for hour_shift in shifts}

#We make a dataset with the lag values
shifted_data = pd.DataFrame(shifted_data)
shifted_data

,lag_24_hour,lag_25_hour,lag_26_hour,lag_27_hour,lag_28_hour,lag_29_hour,lag_30_hour,lag_31_hour,lag_32_hour,lag_33_hour,...,lag_63_hour,lag_64_hour,lag_65_hour,lag_66_hour,lag_67_hour,lag_68_hour,lag_69_hour,lag_70_hour,lag_71_hour,lag_72_hour
datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-05 19:00:00,2.01,2.29,2.33,2.75,5.11,6.95,7.03,6.65,6.37,5.79,...,9.42,8.94,8.56,3.80,2.48,2.89,1.83,1.99,2.94,1.86
2022-12-05 20:00:00,1.86,2.01,2.29,2.33,2.75,5.11,6.95,7.03,6.65,6.37,...,10.09,9.42,8.94,8.56,3.80,2.48,2.89,1.83,1.99,2.94
2022-12-05 21:00:00,3.06,1.86,2.01,2.29,2.33,2.75,5.11,6.95,7.03,6.65,...,10.67,10.09,9.42,8.94,8.56,3.80,2.48,2.89,1.83,1.99


In [11]:
train = pd.concat([train,shifted_data],axis=1)
train

,Luftdruck_STA,Lufttemp_STA,Windgeschw_STA,StampfenbachstrOzon,AnzFahrzeuge_STA,hour_of_day,day_of_week,day_of_year,month,year,...,lag_63_hour,lag_64_hour,lag_65_hour,lag_66_hour,lag_67_hour,lag_68_hour,lag_69_hour,lag_70_hour,lag_71_hour,lag_72_hour
datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 01:00:00,954.0,2.1,4.7,8.51,1320.0,1,5,1,1,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 02:00:00,954.4,2.3,3.2,5.34,919.0,2,5,1,1,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 03:00:00,954.6,2.0,2.5,7.54,617.0,3,5,1,1,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 04:00:00,954.6,2.3,2.2,31.30,445.0,4,5,1,1,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 05:00:00,954.8,2.3,2.9,40.87,371.0,5,5,1,1,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-05 19:00:00,968.6,5.7,3.6,22.99,2781.0,19,1,339,12,2022,...,9.42,8.94,8.56,3.80,2.48,2.89,1.83,1.99,2.94,1.86
2022-12-05 20:00:00,968.9,5.3,2.9,22.99,2204.0,20,1,339,12,2022,...,10.09,9.42,8.94,8.56,3.80,2.48,2.89,1.83,1.99,2.94
2022-12-05 21:00:00,969.0,4.4,3.6,12.72,2209.0,21,1,339,12,2022,...,10.67,10.09,9.42,8.94,8.56,3.80,2.48,2.89,1.83,1.99


In [12]:
from statsmodels.tsa.seasonal import seasonal_decompose
#We add the seasonal component
decomp = seasonal_decompose(df_sta_for["StampfenbachstrOzon"])
train["seasonality"] = decomp.seasonal

In [13]:
#We drop the NaN values
train = train.dropna()

In [14]:
#We create our X_train and y_train sets
X_train = train.drop("StampfenbachstrOzon", axis=1)
y_train = train["StampfenbachstrOzon"]

In [15]:
import numpy as np
from sklearn.metrics import mean_squared_error
#We set the necessary conditions for our walk-forward
history = train
performance_collector = []

In [16]:
from sklearn.ensemble import RandomForestRegressor
#We fit our model
forecast_forest_sta = RandomForestRegressor(n_estimators=300, random_state=1)
forecast_forest_sta.fit(X_train, y_train)

RandomForestRegressor(n_estimators=300, random_state=1)

In [18]:
#We define the dataset for our predictions
hourly_predictions_df = pd.DataFrame(columns=['Day', 'Hour', 'Predicted_Ozon', 'Actual_Ozon'])

In [19]:
for d in test["day_of_year"].unique():
    dat = history.tail(72)  # get last 72 hours of available data
    dat = pd.concat([dat, test[test["day_of_year"] == d]])  # add test data

    shifted_data = {"lag_{}_hour".format(hour_shift): dat["StampfenbachstrOzon"].shift(hour_shift) for hour_shift in shifts}  # lag training data
    shifted_data = pd.DataFrame(shifted_data)

    dat = pd.concat([dat[['Luftdruck_STA', 'Lufttemp_STA', 'Windgeschw_STA', 'StampfenbachstrOzon', 'AnzFahrzeuge_STA', 'hour_of_day', 'day_of_week', 'day_of_year', 'month', 'year']], shifted_data], axis=1)
    
    dat = dat.dropna()  # delete missing data
    
    dat["seasonality"] = decomp.seasonal.head(24).values

    pred_dat = dat.drop("StampfenbachstrOzon", axis=1)

    y_pred = forecast_forest_sta.predict(pred_dat)
    actual = test[test["day_of_year"] == d]["StampfenbachstrOzon"]

    # Evaluate performance using mean squared error for each hour
    for h in range(24):
        actual_hourly = test[(test["hour_of_day"] == h) & (test["day_of_year"] == d)]["StampfenbachstrOzon"]
        y_pred_hourly = y_pred[h::24]  # Extract predictions for the current hour from the 24-hour prediction
        print(f'Day {d}, Hour {h}, y_pred_hourly: {y_pred_hourly.tolist()}')
        hourly_predictions_df = pd.concat([hourly_predictions_df, pd.DataFrame({'Day': [d], 'Hour': [h], 'Predicted_Ozon': [y_pred_hourly.tolist()], 'Actual_Ozon': [actual_hourly.tolist()]})], ignore_index=True)
        
    performance_collector.append(mean_squared_error(actual,y_pred, squared=False))

    history = pd.concat([history, dat], ignore_index=True) #append funktionierte nicht

Day 340, Hour 0, y_pred_hourly: [5.544266666666666]
Day 340, Hour 1, y_pred_hourly: [10.028333333333329]
Day 340, Hour 2, y_pred_hourly: [10.966766666666663]
Day 340, Hour 3, y_pred_hourly: [11.519833333333331]
Day 340, Hour 4, y_pred_hourly: [7.456400000000005]
Day 340, Hour 5, y_pred_hourly: [13.61971666666666]
Day 340, Hour 6, y_pred_hourly: [13.285616666666654]
Day 340, Hour 7, y_pred_hourly: [16.452749999999995]
Day 340, Hour 8, y_pred_hourly: [15.361583333333325]
Day 340, Hour 9, y_pred_hourly: [12.203449999999993]
Day 340, Hour 10, y_pred_hourly: [20.73306666666669]
Day 340, Hour 11, y_pred_hourly: [19.962538095238088]
Day 340, Hour 12, y_pred_hourly: [23.84323333333333]
Day 340, Hour 13, y_pred_hourly: [25.100877777777747]
Day 340, Hour 14, y_pred_hourly: [24.17193333333332]
Day 340, Hour 15, y_pred_hourly: [22.943438095238083]
Day 340, Hour 16, y_pred_hourly: [22.81901587301587]
Day 340, Hour 17, y_pred_hourly: [20.608771428571437]
Day 340, Hour 18, y_pred_hourly: [20.85956984

Day 349, Hour 0, y_pred_hourly: [20.86043333333333]
Day 349, Hour 1, y_pred_hourly: [21.700699999999998]
Day 349, Hour 2, y_pred_hourly: [20.856066666666674]
Day 349, Hour 3, y_pred_hourly: [20.252333333333304]
Day 349, Hour 4, y_pred_hourly: [19.258033333333337]
Day 349, Hour 5, y_pred_hourly: [20.38703333333331]
Day 349, Hour 6, y_pred_hourly: [19.719099999999994]
Day 349, Hour 7, y_pred_hourly: [21.43890000000001]
Day 349, Hour 8, y_pred_hourly: [16.857766666666656]
Day 349, Hour 9, y_pred_hourly: [17.530933333333337]
Day 349, Hour 10, y_pred_hourly: [18.206733333333332]
Day 349, Hour 11, y_pred_hourly: [18.09696666666666]
Day 349, Hour 12, y_pred_hourly: [17.797499999999996]
Day 349, Hour 13, y_pred_hourly: [24.305899999999983]
Day 349, Hour 14, y_pred_hourly: [24.558333333333323]
Day 349, Hour 15, y_pred_hourly: [23.986033333333317]
Day 349, Hour 16, y_pred_hourly: [24.12219999999999]
Day 349, Hour 17, y_pred_hourly: [23.4251]
Day 349, Hour 18, y_pred_hourly: [24.647466666666674]


Day 359, Hour 0, y_pred_hourly: [30.926777777777776]
Day 359, Hour 1, y_pred_hourly: [27.36622222222221]
Day 359, Hour 2, y_pred_hourly: [27.667066666666656]
Day 359, Hour 3, y_pred_hourly: [31.363955555555574]
Day 359, Hour 4, y_pred_hourly: [33.21652222222226]
Day 359, Hour 5, y_pred_hourly: [32.987583333333376]
Day 359, Hour 6, y_pred_hourly: [30.975599999999996]
Day 359, Hour 7, y_pred_hourly: [30.919633333333326]
Day 359, Hour 8, y_pred_hourly: [31.75287222222223]
Day 359, Hour 9, y_pred_hourly: [40.685]
Day 359, Hour 10, y_pred_hourly: [35.347133333333325]
Day 359, Hour 11, y_pred_hourly: [34.04025555555557]
Day 359, Hour 12, y_pred_hourly: [34.061399999999985]
Day 359, Hour 13, y_pred_hourly: [31.85586666666666]
Day 359, Hour 14, y_pred_hourly: [28.980105555555546]
Day 359, Hour 15, y_pred_hourly: [32.759105555555514]
Day 359, Hour 16, y_pred_hourly: [28.862549999999942]
Day 359, Hour 17, y_pred_hourly: [32.286549999999956]
Day 359, Hour 18, y_pred_hourly: [30.496116666666634]
D

Day 2, Hour 0, y_pred_hourly: [18.07006666666664]
Day 2, Hour 1, y_pred_hourly: [12.760633333333327]
Day 2, Hour 2, y_pred_hourly: [18.222022222222233]
Day 2, Hour 3, y_pred_hourly: [18.247033333333317]
Day 2, Hour 4, y_pred_hourly: [15.537766666666672]
Day 2, Hour 5, y_pred_hourly: [9.820299999999994]
Day 2, Hour 6, y_pred_hourly: [23.346888888888916]
Day 2, Hour 7, y_pred_hourly: [22.881483333333346]
Day 2, Hour 8, y_pred_hourly: [20.05271428571431]
Day 2, Hour 9, y_pred_hourly: [15.168228571428571]
Day 2, Hour 10, y_pred_hourly: [23.10758571428573]
Day 2, Hour 11, y_pred_hourly: [29.847616666666692]
Day 2, Hour 12, y_pred_hourly: [32.579233333333356]
Day 2, Hour 13, y_pred_hourly: [25.410266666666665]
Day 2, Hour 14, y_pred_hourly: [42.20522777777778]
Day 2, Hour 15, y_pred_hourly: [28.79446666666666]
Day 2, Hour 16, y_pred_hourly: [28.85056666666669]
Day 2, Hour 17, y_pred_hourly: [32.081033333333295]
Day 2, Hour 18, y_pred_hourly: [26.70655555555556]
Day 2, Hour 19, y_pred_hourly:

Day 10, Hour 7, y_pred_hourly: [48.46153333333332]
Day 10, Hour 8, y_pred_hourly: [45.53851111111108]
Day 10, Hour 9, y_pred_hourly: [43.158844444444455]
Day 10, Hour 10, y_pred_hourly: [47.06759999999998]
Day 10, Hour 11, y_pred_hourly: [53.99017777777778]
Day 10, Hour 12, y_pred_hourly: [58.826944444444464]
Day 10, Hour 13, y_pred_hourly: [59.44866666666664]
Day 10, Hour 14, y_pred_hourly: [58.73450000000002]
Day 10, Hour 15, y_pred_hourly: [57.937966666666675]
Day 10, Hour 16, y_pred_hourly: [48.63317777777777]
Day 10, Hour 17, y_pred_hourly: [49.547733333333305]
Day 10, Hour 18, y_pred_hourly: [48.55761111111108]
Day 10, Hour 19, y_pred_hourly: [55.21969444444449]
Day 10, Hour 20, y_pred_hourly: [58.67000000000002]
Day 10, Hour 21, y_pred_hourly: [61.322477777777785]
Day 10, Hour 22, y_pred_hourly: [58.07453888888892]
Day 10, Hour 23, y_pred_hourly: [58.54472222222224]
Day 11, Hour 0, y_pred_hourly: [59.633277777777806]
Day 11, Hour 1, y_pred_hourly: [59.42656666666668]
Day 11, Hou

...

Day 300, Hour 0, y_pred_hourly: [51.81518518518518]
Day 300, Hour 1, y_pred_hourly: [48.29180000000002]
Day 300, Hour 2, y_pred_hourly: [48.16683333333337]
Day 300, Hour 3, y_pred_hourly: [41.22371111111109]
Day 300, Hour 4, y_pred_hourly: [41.23184444444444]
Day 300, Hour 5, y_pred_hourly: [45.58361111111107]
Day 300, Hour 6, y_pred_hourly: [58.558244444444405]
Day 300, Hour 7, y_pred_hourly: [56.59149999999993]
Day 300, Hour 8, y_pred_hourly: [59.69985000000002]
Day 300, Hour 9, y_pred_hourly: [57.93381111111106]
Day 300, Hour 10, y_pred_hourly: [57.16179999999998]
Day 300, Hour 11, y_pred_hourly: [56.47304444444441]
Day 300, Hour 12, y_pred_hourly: [47.140822222222226]
Day 300, Hour 13, y_pred_hourly: [47.36350000000005]
Day 300, Hour 14, y_pred_hourly: [53.32255555555559]
Day 300, Hour 15, y_pred_hourly: [52.47344444444449]
Day 300, Hour 16, y_pred_hourly: [44.42451111111112]
Day 300, Hour 17, y_pred_hourly: [34.86275555555559]
Day 300, Hour 18, y_pred_hourly: [34.738922222222236]


Day 308, Hour 0, y_pred_hourly: [37.25701111111111]
Day 308, Hour 1, y_pred_hourly: [42.02487777777777]
Day 308, Hour 2, y_pred_hourly: [45.67653333333334]
Day 308, Hour 3, y_pred_hourly: [47.97414444444443]
Day 308, Hour 4, y_pred_hourly: [55.64386666666667]
Day 308, Hour 5, y_pred_hourly: [47.6368666666667]
Day 308, Hour 6, y_pred_hourly: [46.372366666666686]
Day 308, Hour 7, y_pred_hourly: [35.90064444444442]
Day 308, Hour 8, y_pred_hourly: [29.727766666666668]
Day 308, Hour 9, y_pred_hourly: [28.137600000000003]
Day 308, Hour 10, y_pred_hourly: [35.02156666666668]
Day 308, Hour 11, y_pred_hourly: [33.62781111111113]
Day 308, Hour 12, y_pred_hourly: [35.976244444444504]
Day 308, Hour 13, y_pred_hourly: [37.27106666666666]
Day 308, Hour 14, y_pred_hourly: [39.90195555555554]
Day 308, Hour 15, y_pred_hourly: [46.86720555555557]
Day 308, Hour 16, y_pred_hourly: [36.58684444444445]
Day 308, Hour 17, y_pred_hourly: [34.61313333333334]
Day 308, Hour 18, y_pred_hourly: [47.42947222222221]


Day 315, Hour 2, y_pred_hourly: [52.47129999999999]
Day 315, Hour 3, y_pred_hourly: [47.52380000000003]
Day 315, Hour 4, y_pred_hourly: [47.614100000000015]
Day 315, Hour 5, y_pred_hourly: [49.15089999999996]
Day 315, Hour 6, y_pred_hourly: [53.04873333333327]
Day 315, Hour 7, y_pred_hourly: [52.96013333333331]
Day 315, Hour 8, y_pred_hourly: [54.691333333333326]
Day 315, Hour 9, y_pred_hourly: [54.60063333333327]
Day 315, Hour 10, y_pred_hourly: [54.903477777777695]
Day 315, Hour 11, y_pred_hourly: [53.52677777777774]
Day 315, Hour 12, y_pred_hourly: [50.329711111111095]
Day 315, Hour 13, y_pred_hourly: [47.65698888888891]
Day 315, Hour 14, y_pred_hourly: [52.3180111111111]
Day 315, Hour 15, y_pred_hourly: [52.64982222222222]
Day 315, Hour 16, y_pred_hourly: [50.41974444444448]
Day 315, Hour 17, y_pred_hourly: [50.798900000000096]
Day 315, Hour 18, y_pred_hourly: [42.81930000000006]
Day 315, Hour 19, y_pred_hourly: [44.192322222222245]
Day 315, Hour 20, y_pred_hourly: [43.139166666666

Day 323, Hour 0, y_pred_hourly: [23.33063333333333]
Day 323, Hour 1, y_pred_hourly: [31.047633333333362]
Day 323, Hour 2, y_pred_hourly: [47.49614444444443]
Day 323, Hour 3, y_pred_hourly: [27.348133333333347]
Day 323, Hour 4, y_pred_hourly: [19.608466666666665]
Day 323, Hour 5, y_pred_hourly: [22.18833333333333]
Day 323, Hour 6, y_pred_hourly: [49.430411111111155]
Day 323, Hour 7, y_pred_hourly: [48.68360555555557]
Day 323, Hour 8, y_pred_hourly: [47.79157222222224]
Day 323, Hour 9, y_pred_hourly: [47.02938333333333]
Day 323, Hour 10, y_pred_hourly: [45.97727222222219]
Day 323, Hour 11, y_pred_hourly: [46.600033333333364]
Day 323, Hour 12, y_pred_hourly: [51.007666666666665]
Day 323, Hour 13, y_pred_hourly: [52.56689999999998]
Day 323, Hour 14, y_pred_hourly: [53.57373333333331]
Day 323, Hour 15, y_pred_hourly: [54.311033333333334]
Day 323, Hour 16, y_pred_hourly: [52.501833333333344]
Day 323, Hour 17, y_pred_hourly: [49.931266666666644]
Day 323, Hour 18, y_pred_hourly: [52.5713666666

Day 330, Hour 6, y_pred_hourly: [33.670433333333314]
Day 330, Hour 7, y_pred_hourly: [46.32449999999992]
Day 330, Hour 8, y_pred_hourly: [27.0559]
Day 330, Hour 9, y_pred_hourly: [23.968633333333322]
Day 330, Hour 10, y_pred_hourly: [24.80620000000001]
Day 330, Hour 11, y_pred_hourly: [46.30958333333331]
Day 330, Hour 12, y_pred_hourly: [52.267599999999966]
Day 330, Hour 13, y_pred_hourly: [45.48386666666666]
Day 330, Hour 14, y_pred_hourly: [35.19021111111113]
Day 330, Hour 15, y_pred_hourly: [34.85900000000003]
Day 330, Hour 16, y_pred_hourly: [34.186066666666676]
Day 330, Hour 17, y_pred_hourly: [27.95738888888889]
Day 330, Hour 18, y_pred_hourly: [37.00622222222224]
Day 330, Hour 19, y_pred_hourly: [33.92923333333332]
Day 330, Hour 20, y_pred_hourly: [28.284052380952385]
Day 330, Hour 21, y_pred_hourly: [34.04972698412701]
Day 330, Hour 22, y_pred_hourly: [23.865273015873015]
Day 330, Hour 23, y_pred_hourly: [25.27315238095239]
Day 331, Hour 0, y_pred_hourly: [24.383582539682532]
D

In [20]:
#we print our hourly predictions
hourly_predictions_df

,Day,Hour,Predicted_Ozon,Actual_Ozon
0,340,0,[5.544266666666666],[3.65]
1,340,1,[10.028333333333329],[2.87]
2,340,2,[10.966766666666663],[12.55]
3,340,3,[11.519833333333331],[28.25]
4,340,4,[7.456400000000005],[7.35]
...,...,...,...,...
8659,335,19,[15.701966666666676],[0.0]
8660,335,20,[20.00276666666666],[0.0]
8661,335,21,[20.03916666666665],[0.0]
8662,335,22,[18.156066666666664],[0.0]


In [21]:
#We remove [] from our dataset
hourly_predictions_df = hourly_predictions_df.explode('Predicted_Ozon')
hourly_predictions_df = hourly_predictions_df.explode('Actual_Ozon')

# Reset the index
hourly_predictions_df.reset_index(drop=True, inplace=True)

# Display the DataFrame after modification
print(hourly_predictions_df)

      Day Hour Predicted_Ozon Actual_Ozon
0     340    0       5.544267        3.65
1     340    1      10.028333        2.87
2     340    2      10.966767       12.55
3     340    3      11.519833       28.25
4     340    4         7.4564        7.35
...   ...  ...            ...         ...
8659  335   19      15.701967         0.0
8660  335   20      20.002767         0.0
8661  335   21      20.039167         0.0
8662  335   22      18.156067         0.0
8663  335   23      20.453667         0.0

[8664 rows x 4 columns]


In [23]:
#We select only the last 24h of our dataset
hourly_predictions_df = hourly_predictions_df.iloc[8640:8664]

In [24]:
#we fix the hour format
hourly_predictions_df['Hour'] = hourly_predictions_df['Hour'].apply(lambda x: f'{x:02}:00')

In [28]:
#we drop the octual ozon column
hourly_predictions_df = hourly_predictions_df.drop(columns=['Actual_Ozon'])

In [29]:
#We save the dataset for the app
hourly_predictions_df.to_csv(path + 'app_dataset_forecasting.csv', index=False)